<a href=https://uplimit.com/course/r-for-data-science>
  <img src=https://imgur.com/6Vkc11w.png />
</a>

|        |                                   |                                   |                                 |                                |
| -------| ----------------------------------| -------------------------------- -| --------------------------------| ------------------------------------------|
| Week 1 | [[01] Doing Data Science](https://colab.research.google.com/drive/1aWpmzS36qVc8hNIaSSSHexlodwwyMJCB)           | [[02] Data Science in Action](https://colab.research.google.com/drive/1aWpmzS36qVc8hNIaSSSHexlodwwyMJCB)       | [[03] Importing Data](https://colab.research.google.com/drive/13kHBX3H1F2yx5yXWJ5iFZh48qjEtpLjU)             | [[04] Visualizing Data: Overview](https://colab.research.google.com/drive/1JSeDt-Gneegd5B2KwWTJF_igYPb-xsIv)          |
|        | [[05] Transforming Data: Overview](https://colab.research.google.com/drive/1IMZJFE8WmwjgdYCddvaO386PljKq_PjC)  | [[06] Transforming Data: Manipulate](https://colab.research.google.com/drive/19ClnAH7d30I9PkD9KuZndPmy5hxk3gdi) |                                 |                                           |
| Week 2 | [[07] Transforming Data: Aggregate](https://colab.research.google.com/drive/1zigvePewKpzKQHx2pjABXU6Ms8QzQXj7) | [[08] Transforming Data: Reshape](https://colab.research.google.com/drive/1Ma9VrM9SrkNEeXTSul0Ca8a7GHAeNLzC)   | [[09] Transforming Data: Combine](https://colab.research.google.com/drive/1JUGm1h4YPmCkre-rVUj5Ql7lBS66_xIs) | [[10] Visualizing Data: Grammar of Graphics](https://colab.research.google.com/drive/1YLE3NPp-TO2KUTohtidjzKMeqJVWxYoT) |
|        | [[11] Data Science in Action Again](https://colab.research.google.com/drive/1eWjVK4oEiXdwdP4sT0nOJlmkCMBkfLpz) |                                   |                                 |                                           |
Books | [Hands on Programming with R](https://rstudio-education.github.io/hopr/index.html) | [R for Data Science](https://r4ds.hadley.nz/) | [Elegant Graphics for Data Analysis](https://ggplot2-book.org/)



> 📕 __BEFORE YOU GET STARTED:__
> 1. Click on **Copy to Drive** to copy notebook to your drive.
> 2. Click on **Share** on the top-right corner (of your copied notebook).
> 3. Update sharing settings to **Anyone with link**
> 4. Delete this cell.


## Unisex Names

<img src=https://fivethirtyeight.com/wp-content/uploads/2015/06/unisex.jpg />

There are some names that are used commonly by both the sexes. FiveThirtyEight published a [blog post](https://fivethirtyeight.com/features/there-are-922-unisex-names-in-america-is-yours-one-of-them/) on this topic in 2015.

<img src="https://fivethirtyeight.com/wp-content/uploads/2015/06/flowers-datalab-unisexnames-1.png?w=1220" height=600px />


Wouldn't it be cool if we redid the same analysis, but with more data! Let us dive into the data and figure out the most popular unisex names. Let's assume that a name is considered unisex if more than 33% males and 33% females have that name. We can tweak these thresholds subsequently to see if it reveals a different set of names!

Let us load some R packages that we will need to accomplish this task.

In [ ]:
# Load packages and common utility functions
library(tidyverse)
knitr::opts_chunk$set(collapse = TRUE)
url = "https://gist.githubusercontent.com/ramnathv/e71d3ffe88961fc4ef95642053012384/raw/9dda53562d06009912841ab3538d4aeb90e5ee09/_common.R"
download.file(url, basename(url))
source("_common.R")
install.packages(c("gt", "gtExtras"))
library(gt)
library(gtExtras)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Tracing function "install.packages" in package "utils"

Tracing function "update.packages" in package "utils"

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Inspecting installed packages...

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘V8’, ‘reactR’, ‘prismatic’, ‘bigD’, ‘b

### Import

As always, let's start by importing the data from `data/names.csv.gz`.

In [ ]:
file_name_names <- 'https://raw.githubusercontent.com/ramnathv/corise-r-for-ds/main/data/names.csv.gz'
tbl_names <- readr::read_csv(file_name_names, show_col_types = FALSE)
head(tbl_names)

year,name,sex,nb_births
<dbl>,<chr>,<chr>,<dbl>
1880,Mary,F,7065
1880,Anna,F,2604
1880,Emma,F,2003
1880,Elizabeth,F,1939
1880,Minnie,F,1746
1880,Margaret,F,1578


----

Note that we have two other dataset named [data/namesbystate.csv.gz](data/namesbystate.csv.gz) and [data/frenchnames.csv.gz](data/frenchnames.csv.gz) in the data folder. The first dataset gives you number of births by `name`, `sex`, `year` and US `state`, while the second dataset gives you french babynames. So if you feel bored with this US babynames data, feel free to take an alternate dataset for a spin.

----

### Transform

Our goal is to arrive at data that has

#### Step 1

Let us start by computing the total number of births by `name` and `sex`.

In [ ]:
tbl_names_unisex_1 <- tbl_names |>
  group_by(name, sex) |>
  summarize(
    nb_births = sum(nb_births),
    .groups = 'drop'
  )

head(tbl_names_unisex_1)

name,sex,nb_births
<chr>,<chr>,<dbl>
Aaban,M,120
Aabha,F,51
Aabid,M,16
Aabidah,F,5
Aabir,M,10
Aabriella,F,51


#### Step 2

We can now pivot this data to the wide format so we can have ONE row per name.

In [ ]:
tbl_names_unisex_2 <- tbl_names_unisex_1 |>
  pivot_wider(
    names_from = sex,
    values_from = nb_births,
    names_prefix = 'nb_births_',
    values_fill = 0
  )

head(tbl_names_unisex_2)

name,nb_births_M,nb_births_F
<chr>,<dbl>,<dbl>
Aaban,120,0
Aabha,0,51
Aabid,16,0
Aabidah,0,5
Aabir,10,0
Aabriella,0,51


#### Step 3

The wide format makes it easier to compute the percentage of males and percentage of females for each name. Additionally, let us also add a column for the total number of births across sexes for a given name.

In [ ]:
tbl_names_unisex_3 <- tbl_names_unisex_2 |>
 mutate(
   nb_births_total = nb_births_M + nb_births_F,
   pct_births_M = nb_births_M / nb_births_total,
   pct_births_F = nb_births_F / nb_births_total
 )

head(tbl_names_unisex_3)

name,nb_births_M,nb_births_F,nb_births_total,pct_births_M,pct_births_F
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Aaban,120,0,120,1,0
Aabha,0,51,51,0,1
Aabid,16,0,16,1,0
Aabidah,0,5,5,0,1
Aabir,10,0,10,1,0
Aabriella,0,51,51,0,1


#### Step 4

We defined a unisex name as one where the percentage of males and percentage of females are both >= 33%. We also add an additional threshold for total number of births, so that we don't pick up low-volume idiosyncratic names.

In [ ]:
tbl_names_unisex_4 <- tbl_names_unisex_3 |>
  filter(
    pct_births_M > 0.33,
    pct_births_F > 0.33,
    nb_births_total > 50000
  ) |>
  arrange(desc(nb_births_total))

head(tbl_names_unisex_4)

name,nb_births_M,nb_births_F,nb_births_total,pct_births_M,pct_births_F
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Jessie,110714,168626,279340,0.3963414,0.6036586
Riley,98494,123172,221666,0.4443352,0.5556648
Casey,112422,77093,189515,0.5932090,0.4067910
Jackie,78684,90928,169612,0.4639059,0.5360941
Peyton,50509,80495,131004,0.3855531,0.6144469
Jaime,69610,49914,119524,0.5823935,0.4176065


#### Step 5

Finally, we want to group by `name` and `nb_births` and `nest` the percentages into a single column. This will make it easier to visualize the data.

In [ ]:
tbl_names_unisex_5 <- tbl_names_unisex_4 |>
  select(name, nb_births_total, pct_births_M, pct_births_F) |>
  group_by(name, nb_births_total) |>
  nest() |>
  ungroup()

head(tbl_names_unisex_5)

name,nb_births_total,data
<chr>,<dbl>,<list>
Jessie,279340,"0.3963414, 0.6036586"
Riley,221666,"0.4443352, 0.5556648"
Casey,189515,"0.593209, 0.406791"
Jackie,169612,"0.4639059, 0.5360941"
Johnnie,150859,"0.6749084, 0.3250916"
Peyton,131004,"0.3855531, 0.6144469"


In [ ]:
tbl_names_unisex_5$data[[1]]

pct_births_M,pct_births_F
<dbl>,<dbl>
0.3963414,0.6036586


While we have broken down the transformations into multiple pieces so we could focus on each piece separately, it is often better to write the transformations as a single pipeline as it makes it easier to add, remove, and update steps without having to go back and forth.

In [ ]:
tbl_names_unisex <- tbl_names |>
  # Aggregate births across `name` and `sex`
  group_by(name, sex) |>
  summarize(
    nb_births = sum(nb_births),
    .groups = 'drop'
  ) |>
  # Pivot the table wider
  pivot_wider(
    names_from = sex,
    names_prefix = 'nb_births_',
    values_from = nb_births,
    values_fill = 0
  ) |>
  # Add columns for total births and percentage male and female
  mutate(
    nb_births_total = nb_births_M + nb_births_F,
    pct_births_M = nb_births_M / nb_births_total,
    pct_births_F = nb_births_F / nb_births_total
  ) |>
  # Filter for popular unisex names
  # [At least 33% males and females, > 50000 total births]
  filter(
    pct_births_M > 0.33,
    pct_births_F > 0.33,
    nb_births_total > 50000
  ) |>
  arrange(desc(nb_births_total)) |>
  select(name, nb_births_total, pct_births_M, pct_births_F) |>
  group_by(name, nb_births_total) |>
  nest() |>
  ungroup()

### Visualize

We are all set to visualize the data! We can use `ggplot2` to create some lovely plots from this data. But, we are going to use another R package named `gt` and its companion package `gtExtras` to visualize this data as a table. The `gt` package implements a __grammar of tables__ and you will see how it can create amazing looking tables that sometimes look better than plots.

Let us visualize this data as an html table with a stacked bar chart that summarizes information about unisex names, and can help identify popular names that are used relatively equally for both men and women.


In [ ]:
gt_names_unisex <- tbl_names_unisex |>
  # Create an HTML table using the `gt` package
  gt::gt() |>
  # Label columns with descriptive names
  gt::cols_label(
    name = "Name",
    nb_births_total = "Number of People"
  ) |>
  # Format the values in `nb_births_total` to display as whole numbers
  fmt_number(nb_births_total, decimals = 0) |>
  # Add a table column with a stacked horizontal bar plot
  gtExtras::gt_plt_bar_stack(
    data,
    width = 65,
    labels = c("MALE", "FEMALE"),
    palette= c("#2596be", "#f4ba19"),
    fmt_fn = scales::label_percent()
  ) |>
  # Add useful title and subtitle in the table header
  gt::tab_header(
    title = md("**The Most Common Unisex Names**"),
    subtitle = "Names for which at least one-third the names were male, and
    at least one-third were female, through 2021"
  ) |>
  # Theme the plot using 538's theme
  gtExtras::gt_theme_538()

In [ ]:
gt::gtsave(gt_names_unisex, '/tmp/gt_names_unisex.html')
IRdisplay::display_html(file = '/tmp/gt_names_unisex.html')

<!DOCTYPE html>
 
 
 



 
 
 
 
 
 
 
 The Most Common Unisex Names 
 
 
 Names for which at least one-third the names were male, and
 at least one-third were female, through 2021 
 
 
 Name 
 Number of People 
 MALE || FEMALE 
 
 
 
 Jessie 
 279,340 
 <?xml version='1.0' encoding='UTF-8' ?> 40% 60% 
 Riley 
 221,666 
 <?xml version='1.0' encoding='UTF-8' ?> 44% 56% 
 Casey 
 189,515 
 <?xml version='1.0' encoding='UTF-8' ?> 59% 41% 
 Jackie 
 169,612 
 <?xml version='1.0' encoding='UTF-8' ?> 46.4% 53.6% 
 Peyton 
 131,004 
 <?xml version='1.0' encoding='UTF-8' ?> 39% 61% 
 Jaime 
 119,524 
 <?xml version='1.0' encoding='UTF-8' ?> 58% 42% 
 Kerry 
 98,380 
 <?xml version='1.0' encoding='UTF-8' ?> 50.6% 49.4% 
 Kendall 
 97,190 
 <?xml version='1.0' encoding='UTF-8' ?> 36% 64% 
 Jody 
 87,158 
 <?xml version='1.0' encoding='UTF-8' ?> 36% 64% 
 Frankie 
 76,330 
 <?xml version='1.0' encoding='UTF-8' ?> 54.2% 45.8% 
 Quinn 
 75,474 
 <?xml version='1.0' encoding='UTF-8' ?> 45.5% 54.5% 
 Harley 
 67,464 
 <?xml version='1.0' encoding='UTF-8' ?> 59% 41% 
 Pat 
 66,856 
 <?xml version='1.0' encoding='UTF-8' ?> 40% 60% 
 Skyler 
 66,116 
 <?xml version='1.0' encoding='UTF-8' ?> 61% 39% 
 Emerson 
 54,625 
 <?xml version='1.0' encoding='UTF-8' ?> 51.8% 48.2% 
 Tommie 
 51,859 
 <?xml version='1.0' encoding='UTF-8' ?> 66% 34% 
 Emery 
 50,366 
 <?xml version='1.0' encoding='UTF-8' ?> 38% 62% 
 Rowan 
 50,053 
 <?xml version='1.0' encoding='UTF-8' ?> 65% 35%

A detailed explanation of the code is provided below. Note that you can always use the `?` in R to learn more about a specific function.

* `tbl_names_unisex` is a data frame that contains information about unisex names.
* `gt::gt()` is a function that creates a data table from the data frame. The resulting table will include columns for the variables in the data frame, and will have various formatting options applied based on the other functions in the code.
* `gtExtras::gt_plt_bar_stack()` is a function that adds a stacked bar chart to the table. The chart shows the percentage of births by sex for each name, and the bars are colored using the specified palette. The `fmt_fn` argument is used to format the y-axis labels as percentages.
  * `data` is the data frame to use for the chart.
  * `width` specifies the width of the chart.
  * `labels` is a vector of length 2 that specifies the labels for the two bars (male and female).
  * `palette` is a vector of length 2 that specifies the colors for the two bars.
  * `fmt_fn` is a function that is used to format the y-axis labels.
* `fmt_number()` is a function that formats one or more columns in the data table as numbers with a specified number of decimal places.
  * `columns` specifies the columns to format.
  * `decimals` specifies the number of decimal places to include.
* `gt::tab_header()` is a function that sets the title and subtitle for the table.
  * `title` is the title of the table, formatted using Markdown syntax.
  * `subtitle` is the subtitle of the table.
* `gtExtras::gt_theme_538()` is a function that applies a specific theme to the table, which includes various formatting options for fonts, colors, and layout.
* `gt::gtsave()` saves the html to file.
* `IRdisplay::display_html()` displays the html in a notebook.

## Unisex Names Trends

What if we wanted to do one better than the 538 blog post? Well, in addition to showing the most popular unisex names, we could also display trends in the percentage over time.

### Transform

Transforming Data is a lot like cooking. Everyone uses the same tools, but depending on the recipe you follow, you could end up making dishes that taste very different. Our first step here is to get a table of unisex names just like we did before. However, this time, we are going to use a different recipe.

#### Step 1

Begin by grouping the data by `name` and `sex` and computing the total number of births for each `name` and `sex` combination.

In [ ]:
tbl_names_unisex_v2_1 <- tbl_names |>
  # Group by name and sex
  group_by(name, sex) |>
  # Summarize the total number of births, drop the grouping
  summarize(nb_births = sum(nb_births), .groups = 'drop')

head(tbl_names_unisex_v2_1)

name,sex,nb_births
<chr>,<chr>,<dbl>
Aaban,M,120
Aabha,F,51
Aabid,M,16
Aabidah,F,5
Aabir,M,10
Aabriella,F,51


#### Step 2

Next, let us group the data by `name`, and compute the total number of births for each `name`, and the percentage of births for each `sex`.

In [ ]:
tbl_names_unisex_v2_2 <- tbl_names_unisex_v2_1 |>
  # Group by name
  group_by(name) |>
  # For each name, add NEW columns with number and pct of births
  mutate(
    nb_births_total = sum(nb_births),
    pct_births = nb_births / nb_births_total
  ) |>
  # Remove the grouping
  ungroup()

head(tbl_names_unisex_v2_2)

name,sex,nb_births,nb_births_total,pct_births
<chr>,<chr>,<dbl>,<dbl>,<dbl>
Aaban,M,120,120,1
Aabha,F,51,51,1
Aabid,M,16,16,1
Aabidah,F,5,5,1
Aabir,M,10,10,1
Aabriella,F,51,51,1


__Understanding the Difference between Mutate and Summarize__

Note that you can use `mutate()` as well as `summarize()` after a `group_by()`.
The key difference is that `summarize()` will collapse the data into ONE row
per group, whereas `mutate()` will keep the number of rows unchanged.

Let us understand the impact of using `summarize()` vs using `mutate()`. We filter
the data for a specific name so it is easier to look at the output and understand
its impact.

When we use summarize, the data is collapsed into ONE row. In this example, we
have a new column `nb_births_total` that summarizes the total number of births
having the name "Casey".

Note that the only columns that survived were the grouping column and the columns
explicitly defined inside the `summarize()`.

In [ ]:
tbl_names_unisex_v2_1 |>
  filter(name == "Casey") |>
  # Group by name
  group_by(name) |>
  # For each name, add NEW columns with number and pct of births
  summarize(
    nb_births_total = sum(nb_births)
  )

name,nb_births_total
<chr>,<dbl>
Casey,189515


On the other hand, when we use `mutate()`, the all existing rows and columns
are preserved, and `nb_births_total` is added as a new column.

In [ ]:
tbl_names_unisex_v2_1 |>
  filter(name == "Casey") |>
  # Group by name
  group_by(name) |>
  # For each name, add NEW columns with number and pct of births
  mutate(
    nb_births_total = sum(nb_births)
  )

name,sex,nb_births,nb_births_total
<chr>,<chr>,<dbl>,<dbl>
Casey,F,77093,189515
Casey,M,112422,189515


#### Step 3

We filter the data to only keep rows

- where `sex` is `M`
- that have >= 50,000 births
- where percentage of births is between 0.33 and 0.67 (this is equivalent to both M and F having more than 33% share)

In [ ]:
tbl_names_unisex_v2_3 <- tbl_names_unisex_v2_2 |>
  filter(sex == "F") |>
  # Keep only names with more than 50,000 births and pct between 0.33 and 0.67
  filter(
    # Filter for pct_births between 0.33 and 0.67
    between(pct_births, 0.33, 0.67),
    # Filter for total births > 50000
    nb_births_total > 50000
  )

head(tbl_names_unisex_v2_3)

name,sex,nb_births,nb_births_total,pct_births
<chr>,<chr>,<dbl>,<dbl>,<dbl>
Casey,F,77093,189515,0.4067910
Emerson,F,26304,54625,0.4815378
Emery,F,31289,50366,0.6212326
Frankie,F,34954,76330,0.4579327
Harley,F,27687,67464,0.4103967
Jackie,F,90928,169612,0.5360941


We can put them together in a single data pipeline.

In [ ]:
tbl_names_unisex_v2 <- tbl_names |>
  # Group by name and sex
  group_by(name, sex) |>
  # Summarize the total number of births, drop the grouping
  summarize(nb_births = sum(nb_births), .groups = 'drop') |>
  # Group by name
  group_by(name) |>
  # For each name, add NEW columns with number and pct of births
  mutate(
    nb_births_total = sum(nb_births),
    pct_births = nb_births / nb_births_total
  ) |>
  # Remove the grouping
  ungroup() |>
  # Keep rows corresponding to Males
  filter(sex == "M") |>
  # Keep only names with more than 50,000 births and pct between 0.33 and 0.67
  filter(
    # Filter for pct_births between 0.33 and 0.67
    between(pct_births, 0.33, 0.67),
    # Filter for total births > 50000
    nb_births_total > 50000
  )

#### Step 4

Let us join `tbl_names` with `tbl_names_unisex` to keep only names that are considered unisex. Then we group the resulting data by `name` and `year` and adds new columns for the total number of births and the percentage of births for each `name` and `year` combination.


In [ ]:
tbl_names_unisex_trends_1 <- tbl_names |>
  # Semi join with `tbl_names_unisex` by name
  semi_join(tbl_names_unisex, by = "name") |>
  # Group by `name` and `year`
  group_by(name, year) |>
  # Add columns for total number of births and pct births
  mutate(
    nb_births_total = sum(nb_births),
    pct_births = nb_births / nb_births_total
  )

head(tbl_names_unisex_trends_1)

year,name,sex,nb_births,nb_births_total,pct_births
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1880,Jessie,F,635,789,0.8048162
1880,Frankie,F,17,17,1.0000000
1880,Tommie,F,10,25,0.4000000
1880,Jessie,M,154,789,0.1951838
1880,Harley,M,70,70,1.0000000
1880,Emery,M,52,52,1.0000000


You can filter the data for a particular name and arrange it by year to
see what the transformations are doing.

In [ ]:
tbl_names_unisex_trends_1 |>
  filter(name == "Casey") |>
  arrange(year) |>
  head()

year,name,sex,nb_births,nb_births_total,pct_births
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1888,Casey,M,6,6,1
1890,Casey,M,5,5,1
1892,Casey,M,7,7,1
1894,Casey,M,9,9,1
1895,Casey,M,5,5,1
1897,Casey,M,5,5,1


#### Step 5

We want to summarize the trends in percentage of males for a given unisex name over the years. We can do this by filtering the data for Males, grouping by `name`, and adding a list column of `pct_births` across the years. Additionally, we add a few more useful summaries including total number of births, percentage of male births and percentage of female births for each name.

In [ ]:
tbl_names_unisex_trends_2 <- tbl_names_unisex_trends_1 |>
  # Keep only rows for Males
  filter(sex == "M") |>
  # Group by name
  group_by(name) |>
  # Summarize for each name
  summarize(
    # Total number of births
    nb_births_total = sum(nb_births_total),
    # List of pct_births across years
    pct_births_by_year = list(pct_births),
    # Percentage of Males
    pct_births_M = sum(nb_births) / sum(nb_births_total),
    # Percentage of Females
    pct_births_F = 1 - pct_births_M,
    .groups = 'drop'
  )

tbl_names_unisex_trends_2 |>
  head(2)

name,nb_births_total,pct_births_by_year,pct_births_M,pct_births_F
<chr>,<dbl>,<list>,<dbl>,<dbl>
Casey,189515,"1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.8863636, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.8529412, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.9166667, 0.8723404, 0.8703704, 1.0000000, 1.0000000, 1.0000000, 0.9072165, 0.8455882, 0.8796992, 0.8953488, 0.8653846, 0.8523810, 0.8740741, 0.8972810, 0.8502415, 0.8638498, 0.8558352, 0.8219462, 0.8207547, 0.8568019, 0.8674497, 0.8502304, 0.8546638, 0.8700787, 0.8528073, 0.8155340, 0.7919656, 0.7802071, 0.7873462, 0.7803188, 0.6870670, 0.6733404, 0.6778997, 0.6959147, 0.7238679, 0.7320925, 0.6621830, 0.6116293, 0.5970676, 0.5925240, 0.5804539, 0.5582440, 0.5765130, 0.5848551, 0.5236571, 0.5229786, 0.5618478, 0.5557640, 0.5589071, 0.5722447, 0.5430786, 0.5520833, 0.5309395, 0.4992407, 0.4913226, 0.4885834, 0.4843060, 0.5313394, 0.5456724, 0.5479211, 0.5757202, 0.5593750, 0.6087165, 0.5887701, 0.5815217, 0.5859607, 0.5904416, 0.5925661, 0.5932773, 0.5814581, 0.6107456, 0.5965848, 0.5970320, 0.6006985, 0.6097272, 0.6495215, 0.6162943, 0.6462830, 0.7030848, 0.7333333",0.5932090,0.4067910
Emerson,54625,"1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.9219858, 0.8888889, 0.8967742, 0.8881119, 0.8561151, 0.5335463, 0.5401929, 0.5555556, 0.5514706, 0.4433400, 0.4489796, 0.4990654, 0.3616822, 0.3049415, 0.3320965, 0.3342873, 0.3862402, 0.3863913, 0.3905670, 0.3897674, 0.3911306, 0.3853178, 0.4010677, 0.3845671, 0.4006673, 0.3894638, 0.4184375, 0.4165305, 0.4142954",0.5184622,0.4815378


#### Step 6

Finally, let us group the data by `name`, `nb_births_total`, and `pct_births_by_year`, and nest the columns `pct_births_M` and `pct_births_F` into a single column names `pct_births`.

As a last step, we can ungroup this data, arrange it in descending order of total births and select only the columns we need.

In [ ]:
tbl_names_unisex_trends_3 <- tbl_names_unisex_trends_2 |>
  # Group by name, total births and pct_births_by_year
  group_by(name, nb_births_total, pct_births_by_year) |>
  # Nest pct_births_M and pct_births_F into a single column
  nest(pct_births = c(pct_births_M, pct_births_F)) |>
  # Ungroup the data
  ungroup() |>
  # Arrange in descending order of total births
  arrange(desc(nb_births_total)) |>
  # Select the relevant columns (`name`, `nb_births_total`, percentages)
  select(name, nb_births_total, pct_births, pct_births_by_year)

tbl_names_unisex_trends_3 |>
  head(2)

name,nb_births_total,pct_births,pct_births_by_year
<chr>,<dbl>,<list>,<list>
Jessie,279340,"0.3963414, 0.6036586","0.1951838, 0.1778607, 0.1923848, 0.1534553, 0.1661972, 0.1750433, 0.1528716, 0.1524288, 0.1746905, 0.1475410, 0.1616365, 0.1686478, 0.1676687, 0.1496503, 0.1854545, 0.1585761, 0.1719219, 0.1642105, 0.1767042, 0.1944858, 0.2186813, 0.1714922, 0.2066539, 0.1889339, 0.2072848, 0.2044767, 0.2057325, 0.2270588, 0.2043754, 0.2464347, 0.2508508, 0.2439024, 0.2521866, 0.2344572, 0.2490814, 0.2697998, 0.2709801, 0.2820141, 0.2822307, 0.2944233, 0.3232364, 0.3199072, 0.3333333, 0.3311746, 0.3169091, 0.3320507, 0.3364380, 0.3382727, 0.3691747, 0.3703291, 0.3773050, 0.3963090, 0.4032106, 0.3735274, 0.3783409, 0.4054352, 0.3893805, 0.4016204, 0.3970588, 0.4313086, 0.4326964, 0.4337651, 0.4189641, 0.4558641, 0.4590747, 0.4594727, 0.4648723, 0.4759964, 0.5055453, 0.4980526, 0.5214724, 0.5124481, 0.5407950, 0.5425963, 0.5667047, 0.5631579, 0.6057637, 0.6270588, 0.6342679, 0.6213592, 0.6470180, 0.6692525, 0.6782357, 0.6937213, 0.6881639, 0.6481069, 0.6552975, 0.6814688, 0.6911197, 0.7046490, 0.6845070, 0.6786034, 0.7045666, 0.6814480, 0.6768953, 0.6632302, 0.6475216, 0.6575053, 0.5815465, 0.5871560, 0.5743455, 0.5085297, 0.5374040, 0.5666179, 0.5246766, 0.5044603, 0.4668058, 0.4876310, 0.4823897, 0.4969298, 0.5290894, 0.5130890, 0.4888060, 0.4940639, 0.4394819, 0.4424302, 0.4714132, 0.4709193, 0.4632035, 0.4401515, 0.4257188, 0.4925249, 0.4913469, 0.5036145, 0.4847199, 0.5378304, 0.5079070, 0.5169738, 0.4974516, 0.5215420, 0.4921212, 0.4486486, 0.4312977, 0.4027431, 0.3569620, 0.3319088, 0.3475177, 0.3228228, 0.2843450, 0.3185550, 0.2846580, 0.2868687"
Riley,221666,"0.4443352, 0.5556648","1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.9720670, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.9644970, 0.9700599, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.9586207, 1.0000000, 1.0000000, 1.0000000, 0.9720670, 0.9691358, 0.9715909, 1.0000000, 1.0000000, 1.0000000, 0.9687500, 1.0000000, 1.0000000, 1.0000000, 0.9668874, 0.9588235, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.9591837, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.9240506, 1.0000000, 1.0000000, 0.9494949, 1.0000000, 1.0000000, 1.0000000, 1.0000000, 0.9183673, 0.9139785, 0.9067797, 0.9380531, 0.9140625, 0.9277778, 0.9101124, 0.9010989, 0.8829268, 0.8461538, 0.8587786, 0.9200000, 0.8387909, 0.8027888, 0.8439716, 0.8098160, 0.8000000, 0.7739791, 0.7515204, 0.7156271, 0.7134715, 0.6774744, 0.6643575, 0.6756070, 0.6292759, 0.6161554, 0.5622870, 0.5727440, 0.5454002, 0.5033694, 0.4900240, 0.4520458, 0.4449978, 0.4650868, 0.4326944, 0.4171717, 0.4019951, 0.3967995, 0.4072723, 0.3738826, 0.3410729, 0.3377072, 0.2674166, 0.1944351, 0.1875559, 0.1965457, 0.1938862, 0.2032630, 0.2166818"


We can throw all the steps together into a single data pipeline.

In [ ]:
tbl_names_unisex_trends <- tbl_names |>
  # Semi join with `tbl_names_unisex` by name
  semi_join(tbl_names_unisex, by = "name") |>
  # Group by `name` and `year`
  group_by(name, year) |>
  # Add columns for total number of births and pct births
  mutate(
    nb_births_total = sum(nb_births),
    pct_births = nb_births / nb_births_total
  ) |>
  # Keep only rows for Males
  filter(sex == "M") |>
  # Group by name
  group_by(name) |>
  # Summarize for each name
  summarize(
    # Total number of births
    nb_births_total = sum(nb_births_total),
    # List of pct_births across years
    pct_births_by_year = list(pct_births),
    # Percentage of Males
    pct_births_M = sum(nb_births) / sum(nb_births_total),
    # Percentage of Females
    pct_births_F = 1 - pct_births_M,
    .groups = 'drop'
  ) |>
  # Group by name, total births and pct_births_by_year
  group_by(name, nb_births_total, pct_births_by_year) |>
  # Nest pct_births_M and pct_births_F into a single column
  nest(pct_births = c(pct_births_M, pct_births_F)) |>
  # Ungroup the data
  ungroup() |>
  # Arrange in descending order of total births
  arrange(desc(nb_births_total)) |>
  # Select the relevant columns (`name`, `nb_births_total`, percentages)
  select(name, nb_births_total, pct_births, pct_births_by_year)

### Visualize

We can use the `gt` and `gtExtras` package just like we did before. In addition to hte stacked bar plot column, we can also add a column of sparklines that show trends in the percentage of males with a given unisex name across the years. This lets us see some fascinating patterns.

In [ ]:
gt_names_unisex_trends <- tbl_names_unisex_trends |>
  # Create an HTML table using the `gt` package
  gt::gt() |>
  # Label columns with descriptive names
  gt::cols_label(
    name = "Name",
    nb_births_total = "Number of People",
    pct_births_by_year = "Percent Males by Year"
  ) |>
  # Format the values in `nb_births_total` to display as whole numebrs
  fmt_number(nb_births_total, decimals = 0) |>
  # Add a table column with a stacked horizontal bar plot
  gtExtras::gt_plt_bar_stack(
    pct_births,
    width = 65,
    labels = c("MALE", "FEMALE"),
    palette= c("#2596be", "#f4ba19"),
    fmt_fn = scales::label_percent()
  ) |>
  # Add a column with a sparkline of trends
  gtExtras::gt_plt_sparkline(pct_births_by_year) |>
  # Add useful title and subtitle in the table header
  gt::tab_header(
    title = md("**The Most Common Unisex Names**"),
    subtitle = "Names for which at least one-third the names were male, and
    at least one-third were female, through 2021"
  ) |>
  # Theme the plot using 538's theme
  gtExtras::gt_theme_538()

In [ ]:
gt::gtsave(gt_names_unisex_trends, '/tmp/gt_names_unisex_trends.html')
IRdisplay::display_html(file = '/tmp/gt_names_unisex_trends.html')

<!DOCTYPE html>
 
 
 



 
 
 
 
 
 
 
 The Most Common Unisex Names 
 
 
 Names for which at least one-third the names were male, and
 at least one-third were female, through 2021 
 
 
 Name 
 Number of People 
 MALE || FEMALE 
 Percent Males by Year 
 
 
 
 Jessie 
 279,340 
 <?xml version='1.0' encoding='UTF-8' ?> 40% 60% 
 <?xml version='1.0' encoding='UTF-8' ?> 0.29 <polyline points='4.19,10.68 4.63,10.87 5.06,10.71 5.50,11.14 5.94,11.00 6.37,10.90 6.81,11.15 7.25,11.15 7.68,10.91 8.12,11.21 8.55,11.05 8.99,10.97 9.43,10.99 9.86,11.18 10.30,10.79 10.73,11.09 11.17,10.94 11.61,11.02 12.04,10.89 12.48,10.69 12.92,10.42 13.35,10.94 13.79,10.56 14.22,10.75 14.66,10.55 15.10,10.58 15.53,10.57 15.97,10.33 16.40,10.58 16.84,10.12 17.28,10.07 17.71,10.15 18.15,10.06 18.59,10.25 19.02,10.09 19.46,9.86 19.89,9.85 20.33,9.73 20.77,9.73 21.20,9.59 21.64,9.28 22.07,9.31 22.51,9.16 22.95,9.19 23.38,9.35 23.82,9.18 24.26,9.13 24.69,9.11 25.13,8.77 25.56,8.76 26.00,8.68 26.44,8.47 26.87,8.40 27.31,8.72 27.74,8.67 28.18,8.37 28.62,8.55 29.05,8.41 29.49,8.46 29.93,8.09 30.36,8.07 30.80,8.06 31.23,8.22 31.67,7.82 32.11,7.78 32.54,7.78 32.98,7.72 33.41,7.60 33.85,7.27 34.29,7.35 34.72,7.10 35.16,7.20 35.60,6.88 36.03,6.86 36.47,6.60 36.90,6.64 37.34,6.17 37.78,5.94 38.21,5.86 38.65,6.00 39.08,5.72 39.52,5.47 39.96,5.37 40.39,5.20 40.83,5.26 41.27,5.70 41.70,5.63 42.14,5.34 42.57,5.23 43.01,5.08 43.45,5.30 43.88,5.37 44.32,5.08 44.75,5.34 45.19,5.39 45.63,5.54 46.06,5.71 46.50,5.60 46.94,6.44 47.37,6.37 47.81,6.52 48.24,7.24 48.68,6.92 49.12,6.60 49.55,7.06 49.99,7.28 50.42,7.70 50.86,7.47 51.30,7.53 51.73,7.37 52.17,7.01 52.61,7.19 53.04,7.46 53.48,7.40 53.91,8.00 54.35,7.97 54.79,7.65 55.22,7.65 55.66,7.74 56.10,7.99 56.53,8.15 56.97,7.41 57.40,7.43 57.84,7.29 58.28,7.50 58.71,6.92 59.15,7.25 59.58,7.15 60.02,7.36 60.46,7.10 60.89,7.42 61.33,7.90 61.77,8.09 62.20,8.40 62.64,8.90 63.07,9.18 63.51,9.01 63.95,9.28 64.38,9.70 64.82,9.33 65.25,9.70 65.69,9.68 ' style='stroke-width: 1.07; stroke-linecap: butt;' /> 
 Riley 
 221,666 
 <?xml version='1.0' encoding='UTF-8' ?> 44% 56% 
 <?xml version='1.0' encoding='UTF-8' ?> 0.2 <polyline points='4.19,1.84 4.63,1.84 5.06,1.84 5.50,1.84 5.94,1.84 6.37,1.84 6.81,1.84 7.25,1.84 7.68,1.84 8.12,1.84 8.55,1.84 8.99,1.84 9.43,1.84 9.86,1.84 10.30,1.84 10.73,1.84 11.17,1.84 11.61,1.84 12.04,1.84 12.48,1.84 12.92,1.84 13.35,1.84 13.79,1.84 14.22,1.84 14.66,1.84 15.10,1.84 15.53,1.84 15.97,1.84 16.40,1.84 16.84,1.84 17.28,1.84 17.71,1.84 18.15,1.84 18.59,1.84 19.02,1.84 19.46,1.84 19.89,1.84 20.33,1.84 20.77,1.84 21.20,1.84 21.64,1.84 22.07,1.84 22.51,1.84 22.95,2.14 23.38,1.84 23.82,1.84 24.26,1.84 24.69,1.84 25.13,1.84 25.56,2.23 26.00,2.17 26.44,1.84 26.87,1.84 27.31,1.84 27.74,1.84 28.18,1.84 28.62,1.84 29.05,2.29 29.49,1.84 29.93,1.84 30.36,1.84 30.80,2.14 31.23,2.18 31.67,2.15 32.11,1.84 32.54,1.84 32.98,1.84 33.41,2.18 33.85,1.84 34.29,1.84 34.72,1.84 35.16,2.20 35.60,2.29 36.03,1.84 36.47,1.84 36.90,1.84 37.34,1.84 37.78,2.29 38.21,1.84 38.65,1.84 39.08,1.84 39.52,1.84 39.96,1.84 40.39,1.84 40.83,1.84 41.27,2.67 41.70,1.84 42.14,1.84 42.57,2.39 43.01,1.84 43.45,1.84 43.88,1.84 44.32,1.84 44.75,2.73 45.19,2.78 45.63,2.86 46.06,2.52 46.50,2.78 46.94,2.63 47.37,2.82 47.81,2.92 48.24,3.12 48.68,3.53 49.12,3.39 49.55,2.72 49.99,3.61 50.42,4.00 50.86,3.55 51.30,3.93 51.73,4.04 52.17,4.32 52.61,4.57 53.04,4.96 53.48,4.99 53.91,5.38 54.35,5.53 54.79,5.40 55.22,5.91 55.66,6.06 56.10,6.65 56.53,6.53 56.97,6.83 57.40,7.30 57.84,7.44 58.28,7.86 58.71,7.94 59.15,7.72 59.58,8.07 60.02,8.24 60.46,8.41 60.89,8.47 61.33,8.35 61.77,8.72 62.20,9.08 62.64,9.12 63.07,9.89 63.51,10.69 63.95,10.77 64.38,10.67 64.82,10.70 65.25,10.59 65.69,10.45 ' style='stroke-width: 1.07; stroke-linecap: butt;' /> 
 Casey 
 189,515 
 <?xml version='1.0' encoding='UTF-8' ?> 59% 41% 
 <?xml version='1.0' encoding='UTF-8' ?> 0.7 <polyline points='4.23,1.84 4.71,1.84 5.20,1.84 5.68,1.84 6.16,1.84 6.65,1.84 7.13,1.84 7.62,1.84 8.10,1.84 8.58,1.